In [1]:
import json
with open('4506612.json', 'r') as f:
    data = json.load(f)
data

{'matchFacts': {'matchId': 4506612,
  'highlights': {'image': 'https://i.ytimg.com/vi/HUBQ9rFeDCc/maxresdefault.jpg',
   'url': 'https://www.youtube.com/watch?v=HUBQ9rFeDCc',
   'source': 'www.youtube.com'},
  'playerOfTheMatch': {'id': 1114979,
   'name': {'firstName': 'Emmanuel',
    'lastName': 'Agbadou',
    'fullName': 'Emmanuel Agbadou'},
   'teamName': 'Wolverhampton Wanderers',
   'teamId': 8602,
   'rating': {'num': '8.5',
    'isTop': {'isTopRating': True, 'isMatchFinished': True}},
   'minutesPlayed': 90,
   'shotmap': [{'id': 2768714957,
     'eventType': 'Miss',
     'teamId': 8602,
     'playerId': 1114979,
     'playerName': 'Emmanuel Agbadou',
     'x': 78.9936893216,
     'y': 45.4020382125,
     'min': 45,
     'minAdded': 4,
     'isBlocked': False,
     'isOnTarget': False,
     'blockedX': None,
     'blockedY': None,
     'goalCrossedY': 28.707142856,
     'goalCrossedZ': 6.3921379432,
     'expectedGoals': 0.020616678521037102,
     'expectedGoalsOnTarget': 0,
  

In [48]:
match_id = data['matchFacts']['matchId']

match_id

4506612

In [62]:
import pandas as pd

shots = data['shotmap']
shots_list = shots['shots']

shots_df = pd.DataFrame(shots_list)
shots_df["match_id"] = match_id
shots_df = shots_df[["match_id", "teamId", "min", "playerName", "eventType", "expectedGoals", "expectedGoalsOnTarget"]]


shots_df

,match_id,teamId,min,playerName,eventType,expectedGoals,expectedGoalsOnTarget
0,4506612,8602,10,Pablo Sarabia,Miss,0.418143,0.000000
1,4506612,9825,19,Kai Havertz,Miss,0.083993,0.000000
2,4506612,9825,24,Kai Havertz,AttemptSaved,0.169441,0.050372
3,4506612,9825,26,Thomas Partey,Miss,0.028795,0.000000
4,4506612,9825,40,Ethan Nwaneri,AttemptSaved,0.168071,0.000000
5,4506612,9825,40,Ethan Nwaneri,AttemptSaved,0.074105,0.000000
6,4506612,8602,45,Emmanuel Agbadou,Miss,0.020617,0.000000
7,4506612,8602,45,Matheus Cunha,AttemptSaved,0.014261,0.000000
8,4506612,9825,50,Gabriel,AttemptSaved,0.063558,0.000000
9,4506612,9825,53,Declan Rice,AttemptSaved,0.038351,0.058102


In [61]:
import numpy as np

events = data['matchFacts']['events']['events']
cols = ['type', 'time', 'nameStr', 'card', 'isHome']

home_team = data['matchFacts']['poll']['oddspoll']['HomeTeamId']
away_team = data['matchFacts']['poll']['oddspoll']['AwayTeamId']

red_card_df = pd.DataFrame(events)
red_card_df = red_card_df[cols]
red_card_df = red_card_df[red_card_df["type"].isin(['Card'])]
red_card_df = red_card_df[(red_card_df["type"] != "Card") | (red_card_df["card"].isin(["Red", "YellowRed"]))]
red_card_df.loc[red_card_df["type"] == "Card", "type"] = "Red Card"

red_card_df["match_id"] = match_id
red_card_df["team_id"] = np.where(red_card_df["isHome"] == True, home_team, away_team)
red_card_df = red_card_df[["match_id", "team_id", "time", "nameStr", "type"]]

red_card_df

,match_id,team_id,time,nameStr,type
2,4506612,9825,43,Myles Lewis-Skelly,Red Card
7,4506612,8602,70,Joao Gomes,Red Card


In [65]:
import pandas as pd

# Extract the data
match_date = data['matchFacts']['infoBox']['Match Date']['utcTime']
home_team = data['matchFacts']['poll']['oddspoll']['HomeTeamId']
away_team = data['matchFacts']['poll']['oddspoll']['AwayTeamId']
league = data['matchFacts']['infoBox']['Tournament']['id']

home_goals = None
away_goals = None
for event in data['matchFacts']["events"]["events"]:
    if event.get("halfStrShort") == "FT":
        home_goals = event.get("homeScore")
        away_goals = event.get("awayScore")
        break

# Create DataFrame row
match_df = pd.DataFrame({
    'match_date': [match_date],
    'home_team': [home_team],
    'home_goals': [home_goals],
    'away_team': [away_team], 
    'away_goals': [away_goals]
})

match_df["match_id"] = match_id
match_df['league_id'] = league

match_df = match_df[["match_id", "league_id", "match_date", "home_team", "home_goals", "away_team", "away_goals"]]

match_df

,match_id,league_id,match_date,home_team,home_goals,away_team,away_goals
0,4506612,47,2025-01-25T15:00:00.000Z,8602,0,9825,1


In [16]:
stats = data['stats']['Periods']['All']['stats']

stats

[{'title': 'Top stats',
  'key': 'top_stats',
  'stats': [{'title': 'Ball possession',
    'key': 'BallPossesion',
    'stats': [52, 48],
    'format': 'integer',
    'type': 'graph',
    'highlighted': 'home'},
   {'title': 'Expected goals (xG)',
    'key': 'expected_goals',
    'stats': ['0.74', '1.00'],
    'format': 'double',
    'type': 'text',
    'highlighted': 'away'},
   {'title': 'Total shots',
    'key': 'total_shots',
    'stats': [9, 9],
    'format': 'integer',
    'type': 'text',
    'highlighted': 'equal'},
   {'title': 'Shots on target',
    'key': 'ShotsOnTarget',
    'stats': [4, 3],
    'format': 'integer',
    'type': 'text',
    'highlighted': 'home'},
   {'title': 'Big chances',
    'key': 'big_chance',
    'stats': [1, 3],
    'format': 'integer',
    'type': 'text',
    'highlighted': 'away'},
   {'title': 'Big chances missed',
    'key': 'big_chance_missed_title',
    'stats': [1, 3],
    'type': 'text',
    'highlighted': 'home'},
   {'title': 'Accurate passe

In [24]:
all_stats = {}

for category in stats:
    category_key = category['key']
    all_stats[category_key] = {}
    
    for stat in category['stats']:
        stat_key = stat['key']
        all_stats[category_key][stat_key] = stat['stats']

# Access like:
print(all_stats['top_stats']['BallPossesion'])  # [52, 48]
print(all_stats['shots']['total_shots'])  # [9, 9]
print(all_stats['expected_goals']['expected_goals_open_play'])  # ['0.67', '0.48']
print(all_stats['passes']['accurate_passes'])  # ['364 (84%)', '329 (83%)']
print(all_stats['defence']['matchstats.headers.tackles'])  # [27, 22]

all_stats

[52, 48]
[9, 9]
['0.67', '0.48']
['364 (84%)', '329 (83%)']
[27, 22]


{'top_stats': {'BallPossesion': [52, 48],
  'expected_goals': ['0.74', '1.00'],
  'total_shots': [9, 9],
  'ShotsOnTarget': [4, 3],
  'big_chance': [1, 3],
  'big_chance_missed_title': [1, 3],
  'accurate_passes': ['364 (84%)', '329 (83%)'],
  'fouls': [20, 10],
  'corners': [1, 5]},
 'shots': {'shots': [None, None],
  'total_shots': [9, 9],
  'ShotsOffTarget': [4, 3],
  'ShotsOnTarget': [4, 3],
  'blocked_shots': [1, 3],
  'shots_woodwork': [0, 0],
  'shots_inside_box': [4, 6],
  'shots_outside_box': [5, 3]},
 'expected_goals': {'expected_goals': ['0.74', '1.00'],
  'expected_goals_open_play': ['0.67', '0.48'],
  'expected_goals_set_play': ['0.07', '0.52'],
  'expected_goals_non_penalty': ['0.74', '1.00'],
  'expected_goals_on_target': ['0.74', '0.72']},
 'passes': {'passes': [433, 397],
  'accurate_passes': ['364 (84%)', '329 (83%)'],
  'own_half_passes': [189, 145],
  'opposition_half_passes': [175, 184],
  'long_balls_accurate': ['20 (43%)', '14 (36%)'],
  'accurate_crosses': ['3 (

In [27]:
import sqlite3
import pandas as pd
import re

# Extract match details
match_id = data['matchFacts']['matchId']
match_date = data['matchFacts']['infoBox']['Match Date']['utcTime']
home_team = data['matchFacts']['poll']['oddspoll']['HomeTeamId']
away_team = data['matchFacts']['poll']['oddspoll']['AwayTeamId']
league = data['matchFacts']['infoBox']['Tournament']['id']

# Get stats
stats = data['stats']['Periods']['All']['stats']

# Create match data dictionary with basic info
match_data = {
    'match_id': match_id,
    'home_team': home_team,
    'away_team': away_team,
    'match_date': match_date,
    'league': league
}

def parse_value_with_percentage(value):
    """
    Parse values like "364 (84%)" into separate number and percentage
    Returns: (value, percentage) or (value, None) if no percentage
    """
    if value is None:
        return None, None
    
    value_str = str(value)
    
    # Check if there's a percentage in parentheses
    match = re.match(r'^(.+?)\s*\((\d+)%\)$', value_str)
    if match:
        return match.group(1).strip(), int(match.group(2))
    else:
        return value, None

# Add all stats as columns with home_ and away_ prefixes
for category in stats:
    for stat in category['stats']:
        stat_key = stat['key']
        stat_values = stat['stats']
        
        if stat_values and len(stat_values) >= 2:
            # Parse home value
            home_val, home_pct = parse_value_with_percentage(stat_values[0])
            match_data[f'home_{stat_key}'] = home_val
            if home_pct is not None:
                match_data[f'home_{stat_key}_pct'] = home_pct
            
            # Parse away value
            away_val, away_pct = parse_value_with_percentage(stat_values[1])
            match_data[f'away_{stat_key}'] = away_val
            if away_pct is not None:
                match_data[f'away_{stat_key}_pct'] = away_pct

# Convert to DataFrame
df = pd.DataFrame([match_data])

print(df.columns)
print(f"\nMatch ID: {match_id}")

# Save to CSV
df.to_csv('match_stats.csv', mode='a', header=not pd.io.common.file_exists('match_stats.csv'), index=False)

# Save to SQLite
#conn = sqlite3.connect('football_stats.db')
#df.to_sql('match_stats', conn, if_exists='append', index=False)
#conn.close()

print("\n✓ Data saved successfully!")

# Display sample stats with percentages
print(f"\nHome Accurate Passes: {df['home_accurate_passes'].iloc[0]} ({df.get('home_accurate_passes_pct', pd.Series([None])).iloc[0]}%)")
print(f"Away Accurate Passes: {df['away_accurate_passes'].iloc[0]} ({df.get('away_accurate_passes_pct', pd.Series([None])).iloc[0]}%)")

Index(['match_id', 'home_team', 'away_team', 'match_date', 'league',
       'home_BallPossesion', 'away_BallPossesion', 'home_expected_goals',
       'away_expected_goals', 'home_total_shots', 'away_total_shots',
       'home_ShotsOnTarget', 'away_ShotsOnTarget', 'home_big_chance',
       'away_big_chance', 'home_big_chance_missed_title',
       'away_big_chance_missed_title', 'home_accurate_passes',
       'home_accurate_passes_pct', 'away_accurate_passes',
       'away_accurate_passes_pct', 'home_fouls', 'away_fouls', 'home_corners',
       'away_corners', 'home_shots', 'away_shots', 'home_ShotsOffTarget',
       'away_ShotsOffTarget', 'home_blocked_shots', 'away_blocked_shots',
       'home_shots_woodwork', 'away_shots_woodwork', 'home_shots_inside_box',
       'away_shots_inside_box', 'home_shots_outside_box',
       'away_shots_outside_box', 'home_expected_goals_open_play',
       'away_expected_goals_open_play', 'home_expected_goals_set_play',
       'away_expected_goals_set_play